# Vanilla Recurrent Neural Networks (RNNs)

Convolutional Neural Networks (CNNs) excel at tasks like image classification, where fixed-size inputs correspond to fixed-size outputs. However, they face challenges with variable-length sequences, such as time series, text sequences, and image sequences. Recurrent Neural Networks (RNNs) come to the forefront as a solution for processing sequential data.

RNNs find application in diverse fields such as speech recognition, music generation, sentiment analysis, video processing, and text analysis and translation. Their ability to handle sequences makes them a powerful tool in capturing temporal dependencies.

## Overview

The term **Vanilla RNN** is often used to refer to the basic form of recurrent neural network with a single hidden layer and without architectural enhancements. Vanilla RNN has a simple architecture consisting of an **input layer**, a recurrent **hidden layer** and an **output layer**.

A basic RNN processes a time series of input data $\boldsymbol X$ by estimating the output $\boldsymbol y_t$ given the input vector $\boldsymbol x_t$ and the hidden state vector $\boldsymbol h_t$. The hidden state is updated at each time step. It acts as a memory of previous time steps allowing the network to capture sequential patterns.

![RNN structure](images/rnn.png)

For instance, consider a natural language processing task where $\boldsymbol X$ is a sequence of words in a sentence, $\boldsymbol x_t$ is the word at position $\boldsymbol t$, and $\boldsymbol y_t$ represents the predicted probability distribution over the vocabulary for the next word in the sequence. RNN learns from the context of previous words, using the hidden state to generate predictions for the next word in the sentence.

````{important}
While we focus on many-to-many paradigm, where sequences are processed, and outputs are generated at each time step, it is important to highlight flexivbility of RNNs to different input-output models. RNNs can process different sequence-to-sequence architectures, such as one-to-one, one-to-many, many-to-one. 

````{admonition} What are examples of applications that align with these models?
:class: dropdown
![RNN flexibility](images/flexibility.png)

**Q.** What sequence processing model do you think is suitable for sentiment analysis tasks?
````
````


## Forward Pass

![RNN forward](images/forward.png)

Consider a minibatch of inputs $\boldsymbol x_t$ $\in\mathbb{R}^{n \times d}$ at time step $\boldsymbol t$. Each row of $\boldsymbol x_t$ corresponds to one example at time step $\boldsymbol t$ within a minibatch of $n$ sequence examples. The weight parameter $\boldsymbol W_{xh}$ $\in \mathbb{R}^{d \times h}$ and bias parameter $\boldsymbol b_h$ are applied to the current input. Additionally, let $\boldsymbol h_t$ $\in \mathbb{R}^{n \times h}$ denote the hidden layer output at time step $\boldsymbol t$. The calculation of the hidden layer output, $\boldsymbol h_t$ at the current time step, $\boldsymbol t$ is determined by:

$$
    \boldsymbol h_t = \phi({\boldsymbol x_tW_{xh}} + {\boldsymbol h_{t-1}W_{hh}} + {\boldsymbol b_h})
$$

Here, $\phi$ is an **[activation function](https://fedmug.github.io/kbtu-ml-book/mlp/activations.html)** of the hidden layer output. In contrast to [MLP](https://fedmug.github.io/kbtu-ml-book/mlp/layers.html), we preserve the hidden layer output $\boldsymbol h_{t-1}$ from the previous time step. By introducing a new weight parameter $\boldsymbol W_{hh}$ $\in \mathbb{R}^{h \times h}$, we define how to use the hidden layer output from the previous time step in the current time step. Since the hidden state at the current time step uses the same definition as the previous time step, the computation involves *recurrence* which is why the model is called *recurrent neural network*.

For time step $\boldsymbol t$, the **output** of the output layer is computed similarly to [MLP](https://fedmug.github.io/kbtu-ml-book/mlp/forward_backward_pass.html):

$$
\boldsymbol y_t = \boldsymbol x_t\boldsymbol W_{hy} + \boldsymbol b_q 
$$

Here, $\boldsymbol y_t \in \mathbb{R}^{n \times q}$ represents the output variable, $\boldsymbol W_{hy} \in \mathbb{R}^{h \times q}$ is the weight parameter, and $\boldsymbol b_q \in \mathbb{R}^{1 \times q}$ is the bias parameter. In the case of a classification problem, the softmax function can be applied to $\boldsymbol y_t$ to compute the probability distribution of the output categories. As you can see, hidden state at the current time step, $\boldsymbol h_t$, does not only participate in computing hidden state at next time step ${\boldsymbol t+1}$, but is also used in output computation at current time step, $\boldsymbol y_t$.

```{note}
RNNs consistently employ the same set of parameters, $\boldsymbol W_{hx}, \boldsymbol W_{hh}, \boldsymbol W_{hy}, \boldsymbol b_h, \boldsymbol b_q$, across different time steps. This parameter reuse ensures that the computational cost of parameterization remains **constant**, irrespective of the number of time steps.
```


## Training Vanilla RNNs

Recurrent neural networks use **backpropagation through time (BPTT)**, which means forwarding through entire sequence to compute **losses**, then backwarding through entire sequence to compute **gradients** and update the weights accordingly. However, this becomes problematic if we want to train a sequence that is extremely long. In practice, an approximation called **truncated BPTT** is used, which is essentially running forward and backward through **chunks of the sequence** instead of the whole sequence.

![RNN forward](images/truncated_bptt.png)

To compute the loss $\mathcal L$, the discrepancy between the output $\boldsymbol y_t$ and the desired target $\boldsymbol {\widehat y_t}$ is evaluated by an objective function of a time step $\boldsymbol t$ as:

$ \mathcal L_t(\boldsymbol {x_t, y_t, W_h, W_y}) =  l({\boldsymbol {\widehat y_t}, y_t}) $


## Gradient Calculation

The gradient computation involves applying the chain rule to compute the gradients of the loss with respect to the parameters $\boldsymbol w_h$ of objective function.

$
\frac{\partial L_t}{\partial h_1} = \frac{\partial L_t}{\partial y_t} \frac{\partial y_t}{\partial h_1} = \frac{\partial L_t}{\partial y_t} \frac{\partial y_t}{\partial h_t} \frac{\partial h_t}{\partial h_{t-1}} \dots \frac{\partial h_2}{\partial h_1}
$

$
\frac{\partial h_t}{\partial h_{t-1}} = \left[1 - \tanh^2(W_{hh}h_{t-1} + W_{hx}x_t + b)\right] W_{hh}
$

Depending on the size of $\boldsymbol W_{hh}$, the gradient can either vanish or explode over time:

For matrix $\boldsymbol W_{hh}$:
  - If the largest singular value < 1: vanishing gradients.
  - If the largest singular value > 1: exploding gradients.
  
To address the exploding gradient problem a technique called radient clipping is used. Gradient clipping imposes a constraint on the magnitude of the gradients, preventing them from exceeding a predefined threshold. If the L2 norm  of the gradients exceeds the threshold, it scales down all gradients proportionally to ensure that the overall norm is within the specified limit.

$ \nabla_{\text{clipped}} = \frac{clip\_value}{\max(clip\_value, \lVert \nabla \rVert}) \cdot \nabla $

where:
- $ \nabla_{\text{clipped}} $ is the clipped gradient vector
- $ clip\_value $ is the specified threshold
- $ \lVert \nabla \rVert $ is the L2 norm of the gradient vector.

Considering more advanced RNN architectures like Long Short-Term Memory (LSTM) or Gated Recurrent Unit (GRU) is a common and effective approach to address the vanishing gradient problem in traditional RNNs.

